# Osprey usage

Osprey is a WPS server that runs stream flow routing model, called [RVIC](https://rvic.readthedocs.io/en/latest/). To get started, first instatiate the client. Here, the client will try to connect to a remote osprey instance using the url parameter.

In [5]:
from birdy import WPSClient
from pkg_resources import resource_filename
import json
import requests
import os

# Ensure we are in the working directory with access to the data
if os.path.basename(os.getcwd()) != "osprey":
    os.chdir('../../../')

url = 'https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/osprey/wps'
osprey = WPSClient(url)

The list of available processes can be diplayed using the ? command (ex. bird?). Furthermore, help for individual processes can be diplayed using the same command (ex. bird.process?).

In [10]:
# NBVAL_IGNORE_OUTPUT
osprey?
osprey.convert?
osprey.parameters?
osprey.convolution?
osprey.full_rvic?

Object `osprey.convert` not found.
Object `osprey.parameters` not found.
Object `osprey.convolution` not found.
Object `osprey.full_rvic` not found.


Type:            WPSClient
String form:     <birdy.client.base.WPSClient object at 0x7f9c04575400>
File:            ~/Desktop/osprey/venv/lib/python3.6/site-packages/birdy/client/base.py
Docstring:      
A Web Processing Service for Climate Data Analysis.

Processes
---------

hello
    Just says a friendly Hello.Returns a literal string output with Hello plus the inputed name.
Class docstring:
Returns a class where every public method is a WPS process available at
the given url.

Example:
    >>> emu = WPSClient(url='<server url>')
    >>> emu.hello('stranger')
    'Hello stranger'
Init docstring: 
Args:
    url (str): Link to WPS provider. config (Config): an instance
    processes: Specify a subset of processes to bind. Defaults to all
        processes.
    converters (dict): Correspondence of {mimetype: class} to convert
        this mimetype to a python object.
    username (str): passed to :class:`owslib.wps.WebProcessingService`
    password (str): passed to :class:`owslib.wps.